In [1]:
import secretflow as sf

import sys
sys.path.append("/home/yejj/GradExperiments/chapter3_exp1")

sf.shutdown()
#init 10 nodes and 1 server
nodes = [f"party{i}" for i in range(10)]
nodes.append('server')

# init nodes
sf.init(nodes, address="local")
# init PYU device
devices = {}
for i in range(10):
    devices[i] = sf.PYU(f"party{i}")
# alice, bob, agg_server = sf.PYU("alice"), sf.PYU("bob"), sf.PYU("server")
agg_server = sf.PYU("server")

2024-02-22 11:15:46,110	INFO worker.py:1538 -- Started a local Ray instance.


In [2]:
from secretflow.utils.simulation.datasets import load_mnist

(x_train, y_train), (x_test, y_test) = load_mnist(parts=[devices[i] for i in range(10)], normalized_x=True, categorical_y=True)
x_train.partition_shape()

/home/yejj/miniconda3/envs/sf/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/yejj/miniconda3/envs/sf/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


{PYURuntime(party0): (6000, 28, 28),
 PYURuntime(party1): (6000, 28, 28),
 PYURuntime(party2): (6000, 28, 28),
 PYURuntime(party3): (6000, 28, 28),
 PYURuntime(party4): (6000, 28, 28),
 PYURuntime(party5): (6000, 28, 28),
 PYURuntime(party6): (6000, 28, 28),
 PYURuntime(party7): (6000, 28, 28),
 PYURuntime(party8): (6000, 28, 28),
 PYURuntime(party9): (6000, 28, 28)}

In [3]:
from secretflow.ml.nn import FLModel
from secretflow.security.aggregation import PlainAggregator
from secretflow.security.aggregation.experiment import LDPAggregator

from utils.model import create_model
num_classes = 10
input_shape = (28, 28, 1)
model = create_model(input_shape, num_classes, "lenet")

plain_aggregator = PlainAggregator(agg_server)

fed_plain_model = FLModel(server=agg_server,
                    device_list=[devices[i] for i in range(10)],
                    model=model,
                    aggregator=plain_aggregator,
                    strategy="fed_avg_w",
                    backend = "tensorflow")

import time
start_time = time.time()
history = fed_plain_model.fit(x_train,
                    y_train,
                    validation_data=(x_test, y_test),
                    epochs=3,
                    sampler_method="batch",
                    batch_size=128,
                    aggregate_freq=1)
plain_time = time.time() - start_time
# print("cost time : ", time.time()-start_time)

2024-02-22 11:15:56.381037: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-02-22 11:15:57.537545: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-02-22 11:15:57.537656: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-02-22 11:15:57.537665: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed proper

(pid=63988) 2024-02-22 11:16:03.050429: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
(pid=63992) 2024-02-22 11:16:03.273573: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
(pid=64173) 2024-02-22 11:16:03.436249: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
(pid=64381) 2024-02-22 11:16:03.935331: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No suc

Epoch 1/3


Train Processing: :   0%|          | 0/47 [00:00<?, ?it/s]

Epoch 2/3


Train Processing: :   2%|▏         | 1/47 [00:00<00:08,  5.17it/s]

Epoch 3/3


Train Processing: :  98%|█████████▊| 46/47 [00:07<00:00,  6.18it/s, {'loss': 0.2863125, 'accuracy': 0.9159, 'val_loss': 0.21190816, 'val_accuracy': 0.9389}]


In [4]:
dp_aggregator = LDPAggregator(agg_server)

fed_dp_model = FLModel(server=agg_server,
                    device_list=[devices[i] for i in range(10)],
                    model=model,
                    aggregator=dp_aggregator,
                    strategy="fed_avg_w",
                    backend = "tensorflow")

start_time = time.time()
history_dp = fed_dp_model.fit(x_train,
                    y_train,
                    validation_data=(x_test, y_test),
                    epochs=3,
                    sampler_method="batch",
                    batch_size=128,
                    aggregate_freq=1)
dp_time = time.time() - start_time
# print("cost time : ", time.time()-start_time)

INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party party0.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party party1.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party party2.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party party3.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party party4.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party party5.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party party6.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> wit

Epoch 1/3


Train Processing: :   0%|          | 0/47 [00:00<?, ?it/s]

Epoch 2/3


Train Processing: :   0%|          | 0/47 [00:00<?, ?it/s]

Epoch 3/3


Train Processing: :  98%|█████████▊| 46/47 [00:08<00:00,  5.68it/s, {'loss': 0.42724982, 'accuracy': 0.8676429, 'val_loss': 0.26196358, 'val_accuracy': 0.9211}]


In [5]:
from secretflow.security.aggregation.experiment.ppb_aggregator import PPBAggregator

he_aggregator = PPBAggregator(agg_server, [devices[i] for i in range(10)])

fed_he_model = FLModel(server=agg_server,
                    device_list=[devices[i] for i in range(10)],
                    model=model,
                    aggregator=he_aggregator,
                    strategy="fed_avg_w",
                    backend = "tensorflow")

start_time = time.time()
history_phe = fed_he_model.fit(x_train,
                    y_train,
                    validation_data=(x_test, y_test),
                    epochs=3,
                    sampler_method="batch",
                    batch_size=128,
                    aggregate_freq=1)
phe_time = time.time() - start_time
# print("cost time : ", time.time()-start_time)

INFO:root:Create proxy actor <class 'secretflow.security.aggregation.experiment.ppb_aggregator._AggregatorOperator'> with party server.
INFO:root:Create proxy actor <class 'secretflow.security.aggregation.experiment.ppb_aggregator._WorkerOperator'> with party party0.
INFO:root:Create proxy actor <class 'secretflow.security.aggregation.experiment.ppb_aggregator._WorkerOperator'> with party party1.
INFO:root:Create proxy actor <class 'secretflow.security.aggregation.experiment.ppb_aggregator._WorkerOperator'> with party party2.
INFO:root:Create proxy actor <class 'secretflow.security.aggregation.experiment.ppb_aggregator._WorkerOperator'> with party party3.
INFO:root:Create proxy actor <class 'secretflow.security.aggregation.experiment.ppb_aggregator._WorkerOperator'> with party party4.
INFO:root:Create proxy actor <class 'secretflow.security.aggregation.experiment.ppb_aggregator._WorkerOperator'> with party party5.
INFO:root:Create proxy actor <class 'secretflow.security.aggregation.exp

Epoch 1/3


Train Processing: :   0%|          | 0/47 [00:00<?, ?it/s]

Epoch 2/3


Train Processing: :   0%|          | 0/47 [00:00<?, ?it/s]

Epoch 3/3


Train Processing: :  98%|█████████▊| 46/47 [10:52<00:14, 14.19s/it, {'loss': 0.29913703, 'accuracy': 0.91197145, 'val_loss': 0.22117715, 'val_accuracy': 0.9362}]


In [6]:
from secretflow.security.aggregation import SPUAggregator
import spu

DEFAULT_SEMI2K_RUNTIME_CONFIG = {
    'protocol': spu.spu_pb2.SEMI2K,
    'field': spu.spu_pb2.FM128,
}

cdef = sf.utils.testing.cluster_def([f"party{i}" for i in range(10)], DEFAULT_SEMI2K_RUNTIME_CONFIG)
spu_device = sf.SPU(cdef)

ss_aggregator = SPUAggregator(spu_device)

fed_ss_model = FLModel(server=agg_server,
                    device_list=[devices[i] for i in range(10)],
                    model=model,
                    aggregator=ss_aggregator,
                    strategy="fed_avg_w",
                    backend = "tensorflow")

start_time = time.time()
history_ss = fed_ss_model.fit(x_train,
                    y_train,
                    validation_data=(x_test, y_test),
                    epochs=3,
                    sampler_method="batch",
                    batch_size=128,
                    aggregate_freq=1)
ss_time = time.time() - start_time
# print("cost time : ", time.time()-start_time)

INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party party0.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party party1.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party party2.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party party3.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party party4.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party party5.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party party6.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> wit

Epoch 1/3


(SPURuntime(device_id=None, party=party2) pid=61521) 2024-02-22 11:51:22.750 [info] [thread_pool.cc:ThreadPool:30] Create a fixed thread pool with size 63
(SPURuntime(device_id=None, party=party0) pid=61384) 2024-02-22 11:51:22.755 [info] [thread_pool.cc:ThreadPool:30] Create a fixed thread pool with size 63
(SPURuntime(device_id=None, party=party3) pid=61522) 2024-02-22 11:51:22.749 [info] [thread_pool.cc:ThreadPool:30] Create a fixed thread pool with size 63
(SPURuntime(device_id=None, party=party8) pid=62199) 2024-02-22 11:51:22.749 [info] [thread_pool.cc:ThreadPool:30] Create a fixed thread pool with size 63
(SPURuntime(device_id=None, party=party4) pid=61547) 2024-02-22 11:51:22.749 [info] [thread_pool.cc:ThreadPool:30] Create a fixed thread pool with size 63
(SPURuntime(device_id=None, party=party6) pid=62196) 2024-02-22 11:51:22.749 [info] [thread_pool.cc:ThreadPool:30] Create a fixed thread pool with size 63
(SPURuntime(device_id=None, party=party1) pid=61506) 2024-02-22 11:51:

Train Processing: :   0%|          | 0/47 [00:00<?, ?it/s]

Epoch 2/3


Train Processing: :   0%|          | 0/47 [00:00<?, ?it/s]

Epoch 3/3


Train Processing: :  98%|█████████▊| 46/47 [08:19<00:10, 10.86s/it, {'loss': 0.3076249, 'accuracy': 0.90994287, 'val_loss': 0.22905947, 'val_accuracy': 0.9331}]


In [7]:
rows = ['plain', 'dp', 'ss', 'he']
cols = [plain_time, dp_time, ss_time, phe_time]


In [8]:
cols
# 绘制柱状图


[30.902023553848267, 34.65807223320007, 1504.8450841903687, 1983.3110427856445]